In [12]:
#!pip install -q ollama

# Import and create a client that points explicitly to the local daemon
from ollama import Client

# Explicit host helps avoid accidental DNS problems
client = Client(host="http://127.0.0.1:11434")   # <- change if your server runs elsewhere

# Pick a model that you have already pulled locally
model_name = "gpt-oss:120b-cloud"

In [13]:
from pathlib import Path
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
PDF_DIR = DATA_DIR / "pdfs_descargados"
pdfs = list(PDF_DIR.glob("*.pdf"))[:239]

In [14]:
from PyPDF2 import PdfReader
import re

pdf = PdfReader(pdfs[1])

def es_indice(texto):
    if not texto:
        return False
    # Palabras clave típicas de un índice
    if re.search(r"\b(ÍNDICE GENERAL)\b", texto.upper()):
        return True
    # Patrón: varias líneas con "texto ... número"
    lineas = texto.split("\n")
    coincidencias = sum(bool(re.search(r"\.+\s*\d+$", l)) for l in lineas)
    return coincidencias > 3 

# Buscar la página del índice
indice_page_num = None
for i, page in enumerate(pdf.pages, start=1):
    texto = page.extract_text()
    if es_indice(texto):
        indice_page_num = i
        print(f"📖 El índice general está en la página {indice_page_num}")
        break

if indice_page_num is None:
    print("❌ No se encontró la página del índice")

📖 El índice general está en la página 3


In [15]:
import json
import time
import logging
from typing import Dict, Any, List

def _ollama_generate(
    *,
    model: str,
    prompt: str,
    stream: bool = False,
    max_tokens: int | None = None,
) -> Dict[str, Any]:
    """
    Llamada directa a Ollama usando `client.generate`.
    Devuelve el objeto JSON tal cual lo entrega Ollama.
    """
    # El SDK de Ollama usa `client.generate` con los mismos parámetros que la REST API.
    # https://github.com/ollama/ollama/blob/main/docs/api.md#generate
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": stream
    }
    if max_tokens is not None:
        payload["max_tokens"] = max_tokens

    # La respuesta de Ollama (cuando stream=False) es una dict con:
    # {"model": "...", "created_at": "...", "response": "...", "done": true, "context": [...], "total_duration": ...}
    return client.generate(**payload)

def extraer_paginas(
    indice: str,
    *,
    modelo: str = model_name,
    max_intentos: int = 3,
    espera_retry: float = 2.0,
    logger: logging.Logger | None = None,
) -> Dict[str, str]:
    """
    Envía el texto del índice a un modelo local de Ollama y devuelve
    un diccionario `{sección: página}`.
    """
    if logger is None:
        logger = logging.getLogger(__name__)

    # -------------------------------------------------
    # 1️⃣ Prompt
    # -------------------------------------------------
    prompt = f"""\  
Extrae únicamente el índice del texto y devuélvelo en *formato JSON válido*.  
No escribas explicaciones, resúmenes, comentarios, ni texto adicional.  
El resultado debe ser solo el JSON, sin bloques de markdown ni comillas triples.  

- Las secciones principales están numeradas con números romanos (I., II., III., ...).  
- Los subíndices son títulos que siguen a una sección principal pero no tienen numeración romana.  
- El JSON debe tener este formato:

{{
    "NOMBRE_DE_SECCIÓN": {{
        "página": "NÚMERO_DE_PÁGINA",
        "subíndices": {{
            "SUBÍNDICE_1": "NÚMERO_DE_PÁGINA",
            "SUBÍNDICE_2": "NÚMERO_DE_PÁGINA"
        }}
    }}
}}

Si una sección no tiene subíndices, omite el campo "subíndices".  

Texto a procesar:
{indice}
"""

    # -------------------------------------------------
    # 2️⃣ Llamada a Ollama con reintentos
    # -------------------------------------------------
    for intento in range(1, max_intentos + 1):
        try:
            # stream=False para obtener la respuesta completa de una vez
            respuesta = _ollama_generate(
                model=modelo,
                prompt=prompt,
                stream=False
            )
            contenido = respuesta.get("response", "")
            break
        except Exception as exc:
            logger.warning(
                f"Intento {intento}/{max_intentos} falló al llamar a Ollama: {exc!r}"
            )
            if intento == max_intentos:
                logger.error("Se agotaron los reintentos. Devolviendo dict vacío.")
                return {}
            time.sleep(espera_retry)

    # -------------------------------------------------
    # 3️⃣ Normalizar la cadena (quitar bloque markdown)
    # -------------------------------------------------
    json_str = (
        contenido.replace("```json", "")
        .replace("```", "")
        .strip()
    )

    # -------------------------------------------------
    # 4️⃣ Parsear JSON
    # -------------------------------------------------
    try:
        datos = json.loads(json_str)
        # Asegurarnos de que todo sea string y sin espacios de más
        datos = {str(k).strip(): str(v).strip() for k, v in datos.items()}
    except json.JSONDecodeError as e:
        logger.error(
            f"No se pudo decodificar el JSON devuelto por Ollama.\n"
            f"Error: {e}\nRespuesta cruda: {json_str}"
        )
        datos = {}

    return datos

In [61]:
indices = list()

for pdf in pdfs:
    pdf1 = PdfReader(pdf)
    indice_page_num = None
    for i, page in enumerate(pdf1.pages, start=1):
        texto = page.extract_text()
        if es_indice(texto):
            indice_page_num = i
            print(f"📖 El índice general está en la página {indice_page_num}")
            break
    texto = page.extract_text()
    indices.append((pdf, extraer_paginas(texto)))

📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


No se pudo decodificar el JSON devuelto por Ollama.
Error: Expecting value: line 1 column 1 (char 0)
Respuesta cruda: **JSON jerárquico del índice**


{
  "ASISTENCIA": {
    "página": "238"
  },
  "APERTURA DE LA SESIÓN": {
    "página": "243"
  },
  "ACTAS": {
    "página": "243"
  },
  "CUENTA": {
    "página": "243",
    "subíndices": {
      "ENVÍO DE PROYECTO A COMISIÓN": "243",
      "FUSIÓN DE PROYECTOS": "243",
      "SALUDO A INVITADOS A LA CÁMARA DE DIPUTADAS Y DIPUTADOS": "244",
      "INTEGRACIÓN DE COMISIÓN MIXTA": "244",
      "MINUTO DE SILENCIO EN MEMORIA DE ALCALDE DE LA COMUNA DE PINTO, DON MANUEL GUZMÁN AEDO, RECIENTEMENTE FALLECIDO": "246"
    }
  },
  "ORDEN DEL DÍA": {
    "página": "247",
    "subíndices": {
      "PRÓRROGA DE VIGENCIA DE ESTADO DE EXCEPCIÓN CONSTITUCIONAL DE EMERGENCIA EN REGIÓN DE LA ARAUCANÍA Y PROVINCIAS DE ARAUCO Y DEL BIOBÍO, REGIÓN DEL BIOBÍO": "247",
      "AUMENTO DE PENAS POR DELITOS COMETIDOS CONTRA FUNCIONARIOS DE CARABINEROS DE CHIL

📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


No se pudo decodificar el JSON devuelto por Ollama.
Error: Expecting value: line 1 column 1 (char 0)
Respuesta cruda: **Índice extraído (JSON jerárquico)**  


{
  "ASISTENCIA": {
    "página": "10"
  },
  "APERTURA DE LA SESIÓN": {
    "página": "15"
  },
  "ACTAS": {
    "página": "15"
  },
  "CUENTA": {
    "página": "15"
  },
  "TABLA": {
    "página": "15",
    "subíndices": {
      "ELECCIÓN DE LOS MIEMBROS DE LA COMISIÓN EXPERTA A QUE SE REFIERE EL ARTÍCULO 145 DE LA CONSTITUCIÓN POLÍTICA DE LA REPÚBLICA": "15"
    }
  },
  "DOCUMENTOS DE LA CUENTA": {
    "página": "",
    "subíndices": {
      "1. Oficio del Senado que comunica la aprobación del proyecto para promover el envejecimiento positivo, el cuidado integral de las personas mayores y el fortalecimiento de la institucionalidad del adulto mayor": "",
      "2. Oficio del Senado que comunica el acuerdo a la solicitud del Presidente de la República para prorrogar la vigencia del estado de excepción constitucional en la Regi

📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


...

Intento 1/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Intento 2/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Intento 3/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Se agotaron los reintentos. Devolviendo dict vacío.


📖 El índice general está en la página 3


Intento 1/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Intento 2/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Intento 3/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Se agotaron los reintentos. Devolviendo dict vacío.


📖 El índice general está en la página 3


Intento 1/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Intento 2/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Intento 3/3 falló al llamar a Ollama: ResponseError("you've reached your hourly usage limit, please upgrade to continue")
Se agotaron los reintentos. Devolviendo dict vacío.


In [64]:
import csv

with open("indice.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Título", "Página"])  # encabezado
    writer.writerows(indices)

In [16]:
import pandas as pd

df = pd.read_csv("indice.csv", header=None)  # sin encabezado

In [22]:
for _, row in df[1:].iterrows():
    pdf = PdfReader(row[0])
    if row[1] == '{}':
        for i, page in enumerate(pdf.pages, start=1):
            texto = page.extract_text()
            if es_indice(texto):
                indice_page_num = i
                print(f"📖 El índice general está en la página {indice_page_num}")
                break
            texto = page.extract_text()
            row[1] = extraer_paginas(texto)

📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


Intento 1/3 falló al llamar a Ollama: ResponseError('upstream error')


📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


Intento 1/3 falló al llamar a Ollama: ResponseError('upstream error')


📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


...

📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


Intento 1/3 falló al llamar a Ollama: ResponseError('upstream error')


📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


Intento 1/3 falló al llamar a Ollama: ResponseError('upstream error')


📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3
📖 El índice general está en la página 3


In [26]:
df

,0,1
0,Título,Página
1,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': ""{'página': '4'}"", 'INSTALACIÓN..."
2,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': ""{'página': '12'}"", 'APERTURA D..."
3,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': ""{'página': '67'}"", 'APERTURA D..."
4,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': ""{'página': '122'}"", 'APERTURA ..."
...,...,...
235,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'I.- ASISTENCIA': '{'página': '2'}', 'II.- AP..."
236,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': '{'página': ''}', 'APERTURA DE ..."
237,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': '{'página': '2'}', 'APERTURA DE..."
238,C:\Users\angel\OneDrive\Documents\U\2025-2\Pro...,"{'ASISTENCIA': '{'página': None}', 'APERTURA D..."


In [27]:
df.to_csv('indice.csv')